- Due Thursday 11/12 no later than 9:00 a.m., send email to datascience@codeup.com
- Submit link to GitHub notebook that asks and answers questions - document the work you do to justify findings
- Compose an email with the answers to the questions/your findings, and in the email, include the link to your notebook in GitHub and attach your slide. 
- You will not present this, so be sure that the details you need your need your leader to convey/understand are clearly communicated in the email. 
- Slide should be like an exec. Summary and be in form to present. 
- Continue using best practices of acquire.py, prepare.py, etc. 
- No modeling to be done, and no need to split the data into train/validate/test. 
- alumni.codeup.com has info about cohorts/dates/names

- 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
- 2. Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 
- 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 
- 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 
- 5. At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 
- 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)? 
- 7. Which lessons are least accessed? 
- 8. Anything else I should be aware of? 

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

In [3]:
df = pd.read_csv('curriculum.txt',
                engine='python',
                 header=None,
                 index_col=False,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',)

In [4]:
df.head()

,0,1,2,3,4,5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [5]:
df.columns = ['date', 'time', 'page_viewed', 'user_id', 'cohort_id', 'ip']

In [6]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719459 entries, 0 to 719458
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         719459 non-null  object 
 1   time         719459 non-null  object 
 2   page_viewed  719458 non-null  object 
 3   user_id      719459 non-null  int64  
 4   cohort_id    674619 non-null  float64
 5   ip           719459 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 32.9+ MB


In [17]:
no_cohort_df = df[df['cohort_id'].isnull()]

In [18]:
no_cohort_df.head()

,date,time,page_viewed,user_id,cohort_id,ip
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61


In [ ]:
**Drop rows with null values and turn 

In [19]:
df.dropna(inplace=True)
df.cohort_id = df.cohort_id.astype('int')
df['date'] = df.date + " " + df.time
df.drop(columns=('time'), inplace=True)
df.date = pd.to_datetime(df.date)
df = df.set_index('date')